# Generate tfLite model for Zebra **FreeFall Event** dectection using Artificial **Neural Network**

### **Import** library

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re

In [ ]:
tf.__version__

### **Download** data into this colab from cloud & **unzip** data

In [ ]:
!curl -L -o DeviceAbuseDataFall.zip https://github.com/arnabalumnus/Zebra/raw/main/python/eventDataFiltered/DeviceAbuseDataFall.zip

In [ ]:
import zipfile
with zipfile.ZipFile('DeviceAbuseDataFall.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

### **Main** method()

In [ ]:
# Create a folder with name `DeviceAbuseDataFall`
# Copy all data .csv file into this folder before running

def preProcess(rowCount):
    trainingDataFolder = "DeviceAbuseDataFall/"

    dirPath = os.path.join(os.getcwd(), trainingDataFolder)
    files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

    fileList = []
    finalDF = pd.DataFrame()
    
    for file in files:
        # Importing the dataset
        orgData = pd.read_csv(dirPath + file)
        data = pd.DataFrame(orgData.iloc[:, 1:4])

        print('Current File :', file, '# Rowcount :', data.shape[0])       
        if data.shape[0] > rowCount:
            fileList.append(file)
            start = 0
            end = rowCount
           
            count = 0
            for i in range(start, end):
                temp = pd.DataFrame(data.iloc[i:i+1, 0:3])
                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        elif (data.shape[0] >= (rowCount / 2)):
            fileList.append(file)
            diff = rowCount - data.shape[0]
            start = 0
            end = rowCount
            count = 0
            countDown = rowCount - data.shape[0]

            for i in range(start, end):
                if (countDown > 0):
                    temp = pd.DataFrame(data.iloc[0:1, 0:3])
                    countDown -= 1
                else:
                    temp = pd.DataFrame(data.iloc[(i - diff):(i + 1 - diff), 0:3])

                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        else:
            print("No. of rows is less than", (rowCount / 2))
           
    fileList = [i.split('_', 1)[0] for i in fileList]
    fileList = [re.sub("\d+", "", i) for i in fileList]
    actualOutput = pd.DataFrame([fileName[:4] for fileName in fileList])
    actualOutput.columns = ["actualOutput"]
    
    finalDF = finalDF.reset_index(drop=True)
    finalDF = pd.concat([finalDF, actualOutput], axis=1)
    
    return finalDF

### Prepare **Data Frame**

In [ ]:
finalDF = preProcess(200)

In [ ]:
print(finalDF)

### Split **X & Y** dataSet

In [ ]:
X=finalDF.iloc[:, :-1].values

In [ ]:
print(X)

In [ ]:
y=finalDF.iloc[:, -1].values

In [ ]:
print(y)

In [ ]:
type(y)

In [ ]:
myset = set(y)
print(myset)

In [ ]:
outcomeDict = { 'HFYP': 0, 'PFXX' : 1, 'FFXP' : 2, 'FFZM' : 3, 'HFXP' : 4, 'FFYP': 5, 'OTXX' : 6 , 'UTXX' : 7}

In [ ]:
print(y)

In [ ]:
y[y=='HFYP'] = 0
y[y=='PFXX'] = 1
y[y=='FFXP'] = 2
y[y=='FFZM'] = 3
y[y=='HFXP'] = 4
y[y=='FFYP'] = 5
y[y=='OTXX'] = 6
y[y=='UTXX'] = 7

In [ ]:
print(y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
#encoded_Y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)


In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(y)

In [ ]:
print(dummy_y)

### Splitting the dataset into the **Training & Test** dataSet


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

### Train model using **Neural Network**

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=1200, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=8, activation='softmax'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size = 5, epochs = 100)

In [ ]:
print(X_test)

In [ ]:
y_pred = ann.predict(X_test)


In [ ]:
print(y_pred)

In [ ]:
index = tf.argmax(y_pred, axis=1)

In [ ]:
index

In [ ]:
outcome = index.numpy()


In [ ]:
outcomeString = encoder.inverse_transform(outcome)

In [ ]:
print(outcomeString)

In [ ]:
print(index)

In [ ]:
type(index)

In [ ]:
#index.

Read from test data set 

In [ ]:
testDataFolder = "DeviceAbuseDataFallTest"
testDataFile = "FF03YPU_2017_08_17_15_40_36  - prefall.csv"
dirPath = os.path.join(os.getcwd(), testDataFolder)

In [ ]:
print(dirPath)

In [ ]:
testDataFileWithPath = os.path.join(dirPath, testDataFile)

In [ ]:
print(testDataFileWithPath)

In [ ]:
testDF = pd.DataFrame()
testData = pd.read_csv(testDataFileWithPath)

In [ ]:
print(testData)

In [ ]:
testData1 = pd.DataFrame(testData.iloc[:, 1:4])

In [ ]:
print(testData1)

In [ ]:
testData2 = pd.DataFrame()
count = 0
for i in range(0, 200):
  temp = pd.DataFrame(testData1.iloc[i:i+1, 0:3])
  temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
  temp = temp.reset_index(drop=True)
               
  if count == 0:
    tempDF = temp
    count = 1
  else:
    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
            
               

In [ ]:
print(tempDF)

### **Predict** event

In [ ]:
prediction_result = ann.predict(tempDF)

In [ ]:
print(prediction_result)

In [ ]:
prediction_Value = tf.argmax(prediction_result, axis=1)

In [ ]:
print(prediction_Value)

### Convert model into TensorFlowLife (**tfLite**) model & save

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(ann)
tflite_model = converter.convert()

In [ ]:
with open('neural_network_fall_data_model.tflite', 'wb') as f:
  f.write(tflite_model)